## Предобработка даннах соревнований "Plant Pathology 2020 - FGVC7"

https://www.kaggle.com/c/plant-pathology-2020-fgvc7/data

Colab

In [14]:
#from google.colab import files

In [15]:
# files.upload()

In [16]:
#ls

Colab + Google Drive

In [17]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

In [18]:
import shutil
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math

from keras.preprocessing import image

In [19]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [20]:
# Каталог с набором данных
data_dir = '/Users/romanivanov/Documents/DataScience/Kaggle/plant-pathology-2020-fgvc7/imagesCroped/'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Часть набора данных для обучения
train_data_portion = 1 - test_data_portion - val_data_portion

In [21]:
#функция детектирования листка и обрезка фото

def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img,100,200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
                
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    
    return emb_img[row_min:row_max, col_min:col_max]

In [22]:
for i in range(0,1821):
    img = image.load_img(os.path.join(data_dir, "Train_" + str(i) + ".jpg"))
    os.remove(os.path.join(data_dir, "Train_" + str(i) + ".jpg"))
    image.save_img(os.path.join(data_dir, "Train_" + str(i) + ".jpg"), edge_and_cut(np.array(img)))

In [23]:
for i in range(0,1821):
    img = image.load_img(os.path.join(data_dir, "Test_" + str(i) + ".jpg"))
    os.remove(os.path.join(data_dir, "Test_" + str(i) + ".jpg"))
    image.save_img(os.path.join(data_dir, "Test_" + str(i) + ".jpg"), edge_and_cut(np.array(img)))

In [5]:
# Функция создания каталога с подкаталогами по названию классов

def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "healthy"))
    os.makedirs(os.path.join(dir_name, "multiple_diseases"))
    os.makedirs(os.path.join(dir_name, "rust"))
    os.makedirs(os.path.join(dir_name, "scab"))

In [6]:
# Создание структуры каталогов для обучающего, проверочного и тестового набора данных
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [7]:
# Создание папки с изображениями по которым нужно построить прогноз
dir_name = 'predict'
if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
os.makedirs(dir_name)

In [8]:
healthyList = list(train[train['healthy'] == 1].index)
multiple_diseasesList = list(train[train['multiple_diseases'] == 1].index)
rustList = list(train[train['rust'] == 1].index)
scabList = list(train[train['scab'] == 1].index)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_id           1821 non-null   object
 1   healthy            1821 non-null   int64 
 2   multiple_diseases  1821 non-null   int64 
 3   rust               1821 non-null   int64 
 4   scab               1821 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 71.3+ KB


In [10]:
print('healthy = ',len(healthyList)/1821)
print('multiple_diseases = ',len(multiple_diseasesList)/1821)
print('rust = ',len(rustList)/1821)
print('scab = ',len(scabList)/1821)

healthy =  0.2833607907742998
multiple_diseases =  0.049972542559033495
rust =  0.3415705656232839
scab =  0.32509610104338277


In [11]:
round((0.283 + 0.0495 + 0.342 + 0.325)*100, 1)

100.0

In [12]:
#код ниже просто ужас - когда-нибудь я (или не я) перепишу его =)

In [13]:
trainHealthyList = healthyList[0:int(len(healthyList)*train_data_portion)]
valHealthyList = healthyList[(int(len(healthyList)*train_data_portion)):-int(len(healthyList)*test_data_portion)]
testHealthyList = healthyList[-int(len(healthyList)*test_data_portion):]

In [14]:
trainMultiple_diseasesList = multiple_diseasesList[0:int(len(multiple_diseasesList)*train_data_portion)]
valMultiple_diseasesList = multiple_diseasesList[(int(len(multiple_diseasesList)*train_data_portion)):-int(len(multiple_diseasesList)*test_data_portion)]
testMultiple_diseasesList = multiple_diseasesList[-int(len(multiple_diseasesList)*test_data_portion):]

In [15]:
trainRustList = rustList[0:int(len(rustList)*train_data_portion)]
valRustList = rustList[(int(len(rustList)*train_data_portion)):-int(len(rustList)*test_data_portion)]
testRustList = rustList[-int(len(rustList)*test_data_portion):]

In [16]:
trainScabList = scabList[0:int(len(scabList)*train_data_portion)]
valScabList = scabList[(int(len(scabList)*train_data_portion)):-int(len(scabList)*test_data_portion)]
testScabList = scabList[-int(len(scabList)*test_data_portion):]

In [17]:
# Функция копирования изображений в заданный каталог. Изображения копируются в отдельные подкаталоги

In [18]:
def copy_images(list_index, source_dir, dest_dir, name_dir):
    for i in list_index:
        shutil.copy2(os.path.join(source_dir, "Train_" + str(i) + ".jpg"), os.path.join(dest_dir, name_dir))

In [19]:
copy_images(trainHealthyList, data_dir, train_dir, 'healthy')
copy_images(valHealthyList, data_dir, val_dir, 'healthy')
copy_images(testHealthyList, data_dir, test_dir, 'healthy')

In [20]:
copy_images(trainMultiple_diseasesList, data_dir, train_dir, 'multiple_diseases')
copy_images(valMultiple_diseasesList, data_dir, val_dir, 'multiple_diseases')
copy_images(testMultiple_diseasesList, data_dir, test_dir, 'multiple_diseases')

In [21]:
copy_images(trainRustList, data_dir, train_dir, 'rust')
copy_images(valRustList, data_dir, val_dir, 'rust')
copy_images(testRustList, data_dir, test_dir, 'rust')

In [22]:
copy_images(trainScabList, data_dir, train_dir, 'scab')
copy_images(valScabList, data_dir, val_dir, 'scab')
copy_images(testScabList, data_dir, test_dir, 'scab')

In [23]:
def copy_images_test(source_dir, dest_dir):
    for i in range(0,1821):
        shutil.copy2(os.path.join(source_dir, "Test_" + str(i) + ".jpg"), os.path.join(dest_dir))

In [24]:
copy_images_test(data_dir,'predict')